## The Challenge

Fund manager interacts with the market through a number of custodians. Custodians hold client assets, but purchase/sale of those assets are under the direction of the fund manager. Fund manager makes orders to align client portfolios with particular strategies or intents, and places orders with relevant custodians.

On a daily basis the FM has a backlog of orders generated either directly from clients or from some internal strategy. The FM goes through a placement process that results in placement of day orders with a number of custodians. (TODO: are we going to model alignment of fund holdings, or individual orders? stategy == holding each security in a fixed fraction of whole portfolio?).

(In the next layer down the custodians each go to the market to fulfill those orders).

At the end of day, the FM receives allocation feed files showing to which extent the day orders were filled and allocated to clients. Allocations get converted to transactions (and therefore to holdings) for each client. 

The shortfall between allocation and outstanding order then forms the basis for the next day's order backlog.

## The Solution

Reconciliation between custodian allocations and outstanding orders depends on the allocations returned by custodians as being tagged with identifiers defined by the FM. Ideally the allocations are tagged with an FM order id, but could be tagged with something coarser, like client account name, or something finer, like day order id. In these cases the FM will need to update its orders with the necessary tags to enable rec (the full LUSID orders domain model will make placements available to record this information if needed).

In this tutorial you will walk through how to set this up from a completely blank LUSID environment by:

1) Setting up a Scope to hold your portfolios

2) Creating your Instrument Universe 

3) Creating client portfolios

4) Setting your initial client holdings

5) Recording your initial outstanding order backlog

The daily cycle then is:

6) Recording the placement of orders with a number of custodians

7) Receive allocations from custodians at end of day

8) Convert allocations to transactions against client portfolios

9) Reconcile allocations against order backlog

*First things first run the cell below to import the libraries and authenticate your LUSID client*

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
#from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json
import os

# Authenticate our user and create our API client
secrets_path = "/tmp/working/examples/secrets.json"
                
api_factory = lusid.utilities.ApiClientFactory(api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.0.1.0


## 1) Setting up a Scope to hold your portfolios

You will need to create a name for the scope that you will use for the fund. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [2]:
# Give the scope a unique identifier
scope_id = import_data.create_scope_id()
# Give the scope a descriptive name
scope_name = 'UK_Equities_Fund'
# Join the two together to get the full scope name
scope = '{}_{}'.format(scope_name, scope_id)
prettyprint.heading('Scope', scope)

Scope: UK_Equities_Fund_3853-f5ff-c1db-74


## 2) Creating your Instrument Universe 

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/instruments).

*Run the cell below to import your instrument universe*

In [3]:
# Import your instrument universe from a CSV file
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,nan
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.00
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.88
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,nan
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,nan
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,nan
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,nan
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,nan


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument)

As part of this definition you will attach identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Schemes Should I Use With LUSID?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid)

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [4]:
# Initialise a dictionary to hold your instrument definitions
instrument_definitions = {}

# Set the mapping between your identifier columns in the CSV and the available LUSID identifiers
identifier_columns = {
    'isin': 'Isin',
    'figi': 'Figi',
    'ticker': 'Ticker',
    'client_internal': 'ClientInternal'
}

# Iterate over your instrument universe
for index, instrument in instrument_universe.iterrows():

    # Initialise your set of identifiers for this instrument
    identifiers = {}
    
    # Populate your set of identifiers
    for identifier_column, identifier_lusid in identifier_columns.items():
        identifiers[identifier_lusid] = models.InstrumentIdValue(
            value=instrument[identifier_column])
        
    # Create the definition for your instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers
    )

# Call LUSID to upsert your instrument defintions as a batch
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    instruments=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
0,Apple_Nasdaq_AAPL,BBG000B9XVV8,LUID_UYPGX8M3
1,USTreasury_2.00_2021,BBG00FN3B5K8,LUID_H5TPBDJU
2,Amazon_Nasdaq_AMZN,BBG000BVPXP1,LUID_B8SMJ2EN
3,ExpressScripts_NYSE_ESRX,BBG000C16621,LUID_JPI1NK0O
4,Cigna_NYSE_CI,BBG00KXXK940,LUID_KM6EI6OL
5,TrinityIndustries_NYSE_TRN,BBG000BVL406,LUID_VP6OG7JR
6,Arcosa_NYSE_ACA,BBG00JGMWFQ5,LUID_AZDHHQDX
7,USTreasury_6.875_2025,BBG000DQQNJ8,LUID_CTA5G9WD
8,Trex_NYSE_TREX,BBG000BTGM43,LUID_H2NN0PTU


## 3) Creating client portfolios

To create a portfolio you need to give it a name and code - we'll create one for each client.

*Run the cell below to give your client portfolios a name and code*

In [5]:
# Define portfolio ids
portfolios = {'Alice': 'portfolio-' + str(uuid.uuid4()), 'Bob': 'portfolio-' + str(uuid.uuid4())}

for client in portfolios:
    print(client + " " + portfolios[client])

Alice portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Bob portfolio-c342b70c-5ed0-4f5b-a294-196d3500c0b1


Now that you have decided on the name and unique code for your portfolio you can create them. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/portfolios).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 365 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [6]:
# Arbitrary portfolio creation date for illustration
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

for client in portfolios:
    # Create the request to add your portfolio to LUSID
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=client,
        code=portfolios[client],
        base_currency='USD',
        description='The portfolio for client ' + client,
        created=portfolio_creation_date)

    # Call LUSID to create your portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        transaction_portfolio=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Portfolio Effective From: 2019-03-30 14:17:52.006557+00:00
Portfolio Created On: 2020-03-30 07:05:26.660338+00:00

Portfolio Created
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-c342b70c-5ed0-4f5b-a294-196d3500c0b1
Portfolio Effective From: 2019-03-30 14:17:52.006557+00:00
Portfolio Created On: 2020-03-30 07:05:26.712482+00:00



## 4) Setting initial client holdings

Now that you have your instrument universe populated and client portfolios you can load your initial holdings into each. In this case you will import your holdings from a CSV file.

*Run the cell below to import your take on balances*

In [7]:
#Import your holdings
holdings = pd.read_csv('data/fundmanager-initialholdings.csv')
holdings.head(10)

,client,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Alice,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Alice,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,Alice,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,Alice,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD
4,Bob,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,4000,1550.00,7750000.00,USD
5,Bob,Apple_Nasdaq_AAPL,imd_35345375,US0378331005,BBG000B9XVV8,350000,190.00,9417730.00,USD
6,Bob,USTreasury_2.00_2021,imd_34535363,US912828U816,BBG00FN3B5K8,1433,99.25,12063142.75,USD
7,Bob,USTreasury_6.875_2025,imd_34534512,US912810EV62,BBG000DQQNJ8,5643,140.98,13878635.12,USD


You can add these holdings to LUSID by setting the holdings on your portfolios. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The Effect of Holding Adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio). These holdings will be effective as of 4 days ago. This gives you some room to add recent transactions and conduct valuations.

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set the holdings on your client portfolios*

In [8]:
# Set the effective date of these holdings to be 4 days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

# Iterate over your holdings
for index, holding in holdings.iterrows():
    
    client = holding['client']
    holding_adjustments = []
    
    # Create a holding adjustment for this holding
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                'Instrument/default/Figi': holding['figi']},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=holding['quantity'],
                    cost=models.CurrencyAndAmount(
                        amount=holding['total_cost'],
                        currency=holding['currency']),
                    portfolio_cost=holding['total_cost'],
                    price=holding['unit_cost'])
            ]
        )
    )
    
    # Call LUSID to set your holdings 
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).adjust_holdings(
        scope=scope,
        code=portfolios[client],
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    # Pretty print the response 
    prettyprint.set_holdings_response(response, scope, portfolios[client])

Holdings Successfully Set for Portfolio
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Holdings Effective From: 2019-03-30 14:17:52.006557+00:00
Holdings Created On: 2020-03-30 07:05:26.799735+00:00

Holdings Successfully Set for Portfolio
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Holdings Effective From: 2019-03-30 14:17:52.006557+00:00
Holdings Created On: 2020-03-30 07:05:26.853576+00:00

Holdings Successfully Set for Portfolio
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Holdings Effective From: 2019-03-30 14:17:52.006557+00:00
Holdings Created On: 2020-03-30 07:05:26.906133+00:00

Holdings Successfully Set for Portfolio
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Holdings Effective From: 2019-03-30 14:17:52.006557+00:00
Holdings Created On: 2020-03-30 07:05:26.959375+00:00

Holdings

## 5) Recording the order backlog

In [9]:
#Define outstanding orders
orders = pd.read_csv('data/fundmanager-orders.csv')
orders.head(10)

,client,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side
0,Alice,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,574,1600,USD,ORD001,buy
1,Alice,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,321,200,USD,ORD002,sell
2,Alice,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,45000,80,USD,ORD003,buy limit
3,Alice,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150,USD,ORD004,sell limit
4,Bob,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,574,1600,USD,ORD005,buy
5,Bob,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,321,200,USD,ORD006,sell
6,Bob,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,45000,80,USD,ORD007,buy limit
7,Bob,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150,USD,ORD008,sell limit


In [10]:
#Upsert orders
for index, order in orders.iterrows():
    
    client = order['client']
    order_requests = []

    order_requests.append(
        models.OrderRequest(
            code=order['order_id'],
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']},
            properties={},
            order_book=models.ResourceId(
                scope=scope,
                code='orderbook',
                ),
            portfolio=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
    order_set = models.OrderSetRequest(
        order_requests=order_requests
        )

    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        scope=scope,
        request=order_set
        )
    
    prettyprint.upsert_orders_response(response, scope)

Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD001
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 574
Side: buy
LUSID instrument: LUID_B8SMJ2EN
Instrument/default/Figi: BBG000BVPXP1


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD002
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 321
Side: sell
LUSID instrument: LUID_UYPGX8M3
Instrument/default/Figi: BBG000B9XVV8


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD003
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 45000
Side: buy limit
LUSID instrument: LUID_H5TPBDJU
Instrument/default/Figi: BBG00FN3B5K8


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD004
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 32000
Side: sell limit
LUSID instrument: LUID_CTA5G9WD
Instrument/default/Figi: B

## 6) Recording placement of orders with custodians

Once the backlog of orders has been established, the FM goes through a placement process in which orders are blocked together and placed with specific custodians.

Alice has funds with GS, while Bob has funds with MS.

Placements in this case would see ORD[001 - 004] placed with GS, and ORD[005 - 008] placed with MS. We can tag our orders to reflect the placement.

In [11]:
create_request = lusid.models.CreateDataTypeRequest(
    scope=scope,
    code="CustodianCodes",
    type_value_range="Closed",
    display_name="Available Custodian Tags",
    description="This data type contains the available Custodians which can be used",
    value_type="String",
    acceptable_values=[
        "GS",
        "MS"
    ])

response = api_factory.build(lusid.api.DataTypesApi).create_data_type(
    request=create_request)

print("Data Type Created")
prettyprint.heading("Scope", response.id.scope)
prettyprint.heading("Code", response.id.code)

Data Type Created
Scope: UK_Equities_Fund_3853-f5ff-c1db-74
Code: CustodianCodes


In [12]:
property_request = models.CreatePropertyDefinitionRequest(
    domain='Order',
    scope=scope,
    code='Custodian',
    value_required=False,
    display_name='Custodian',
    data_type_id=models.ResourceId(scope=scope, code='CustodianCodes'))

# Call LUSID to create our new property
property_response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
custodian_property_key = property_response.key

# Pretty print our key
prettyprint.heading('Custodian Property Key', custodian_property_key)

Custodian Property Key: Order/UK_Equities_Fund_3853-f5ff-c1db-74/Custodian


In [13]:
custodians = { 'Alice': 'GS', 'Bob': 'MS' }

for index, order in orders.iterrows():
    
    client = order['client']
    order_requests = []
    
    order_requests.append(
        models.OrderRequest(
            code=order['order_id'],
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']},
            properties={
                     custodian_property_key: models.PerpetualProperty(
                         key=custodian_property_key,
                         value=models.PropertyValue(label_value=custodians[client])
                     )
                 },
            order_book=models.ResourceId(
                scope=scope,
                code='orderbook',
                ),
            portfolio=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
    order_set = models.OrderSetRequest(
        order_requests=order_requests
        )

    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        scope=scope,
        request=order_set
        )
    
    prettyprint.upsert_orders_response(response, scope)

Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD001
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 574
Side: buy
LUSID instrument: LUID_B8SMJ2EN
Instrument/default/Figi: BBG000BVPXP1
Order/UK_Equities_Fund_3853-f5ff-c1db-74/Custodian: GS


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD002
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 321
Side: sell
LUSID instrument: LUID_UYPGX8M3
Instrument/default/Figi: BBG000B9XVV8
Order/UK_Equities_Fund_3853-f5ff-c1db-74/Custodian: GS


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD003
Order Book: orderbook
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 45000
Side: buy limit
LUSID instrument: LUID_H5TPBDJU
Instrument/default/Figi: BBG00FN3B5K8
Order/UK_Equities_Fund_3853-f5ff-c1db-74/Custodian: GS


Orders upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Order: ORD004
Order 

## 7) Receive allocations from custodians at end of day

In [14]:
#Read allocations
allocations = pd.read_csv('data/fundmanager-allocations.csv')
allocations.head(10)

,client,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order
0,Alice,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,300,1600,USD,ALLOC345,ORD001
1,Alice,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,321,200,USD,ALLOC347,ORD002
2,Alice,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,37000,80,USD,ALLOC349,ORD003
3,Alice,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,100,150,USD,ALLOC351,ORD004
4,Bob,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,574,1600,USD,ALLOC353,ORD005
5,Bob,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,300,200,USD,ALLOC355,ORD006
6,Bob,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,43000,80,USD,ALLOC357,ORD007
7,Bob,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150,USD,ALLOC359,ORD008


In [15]:
#Upsert allocations
for index, allocation in allocations.iterrows():
    
    client = allocation['client']
    allocation_requests = []

    allocation_requests.append(
        models.AllocationRequest(
            code=allocation['allocation_id'],
            allocated_order_id=models.ResourceId(
                scope=scope,
                code=allocation['originating_order']
                ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi']},
            properties={},
            portfolio_id=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
    allocation_set = models.AllocationSetRequest(
        allocation_requests=allocation_requests
        )

    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        scope=scope,
        request=allocation_set
        )
    
    prettyprint.upsert_allocations_response(response, scope)

Allocations upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Allocation: ALLOC345
Originating Order: ORD001
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 300
LUSID instrument: LUID_B8SMJ2EN
Instrument/default/Figi: BBG000BVPXP1


Allocations upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Allocation: ALLOC347
Originating Order: ORD002
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 321
LUSID instrument: LUID_UYPGX8M3
Instrument/default/Figi: BBG000B9XVV8


Allocations upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Allocation: ALLOC349
Originating Order: ORD003
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 37000
LUSID instrument: LUID_H5TPBDJU
Instrument/default/Figi: BBG00FN3B5K8


Allocations upserted
Scope: UK_Equities_Fund_3853-f5ff-c1db-74

Allocation: ALLOC351
Originating Order: ORD004
Portfolio: portfolio-3e81ea81-efcf-4f15-a782-92836ad8c18e
Quantity: 100
LUSID instrument: LUID_CTA5G9WD
Instrument/default

## 8) Convert allocations to transactions against client portfolios

## 9) Reconcile allocations against order backlog

In [24]:
# fetch orders
orders_response = api_factory.build(lusid.api.OrdersApi).list_orders(scope)

orders = {}
allocations = {}

for order in orders_response.values:
    orders[order.id.code] = order

# fetch allocations
allocations_response = api_factory.build(lusid.api.AllocationsApi).list_allocations(scope)

for allocation in allocations_response.values:
    allocations[allocation.allocated_order_id.code] = allocation

for order in orders:
    print (order + ' ' + orders[order].lusid_instrument_id + ' ' + str(orders[order].quantity) + ' ' + allocations[order].id.code + ' ' + str(allocations[order].quantity) + ' ' + str(allocations[order].quantity - orders[order].quantity))

ORD001 LUID_B8SMJ2EN 574 ALLOC345 300 -274
ORD002 LUID_UYPGX8M3 321 ALLOC347 321 0
ORD003 LUID_H5TPBDJU 45000 ALLOC349 37000 -8000
ORD004 LUID_CTA5G9WD 32000 ALLOC351 100 -31900
ORD005 LUID_B8SMJ2EN 574 ALLOC353 574 0
ORD006 LUID_UYPGX8M3 321 ALLOC355 300 -21
ORD007 LUID_H5TPBDJU 45000 ALLOC357 43000 -2000
ORD008 LUID_CTA5G9WD 32000 ALLOC359 32000 0


In [ ]:
# upsert updated orders?